# Installation & Maintenance





## Trust the PSGallery Repository

In [ ]:
Set-PSRepository -name PSGallery -InstallationPolicy Trusted

## Install or Update

In [ ]:
if (Get-Module -ListAvailable -Name dbatools) {
    Update-Module -Name dbatools;
}
else {
    Install-Module -Name dbatools -Scope CurrentUser;
}

# Global Variables

In [ ]:
$Sql17 = "flexo\Sql17";
$Sql19 = "flexo\Sql19";

# Backup & Restore
## Basic Backup

**TODO:** about_splatting. Make a whole section about conventions, splatting, etc.

In [ ]:
$BackupPath = "C:\SQL\Backup\";
$BackupParams = @{
    SqlInstance  = $Sql17;
    Path         = $BackupPath;
    Database     = "Satellites";
    CreateFolder = $true;
}
$BackupResult = Backup-DbaDatabase @BackupParams;
$BackupResult | Format-List -Property *;
# Save this for later
$SingleBackupFile = $BackupResult.BackupPath;
Get-ChildItem $BackupPath | Format-Table -AutoSize;
Get-ChildItem "C:\SQL\Backup\Satellites" -Recurse | Sort-Object -Property LastWriteTime -Descending;

## More Advanced

- Copy-only
- Multiple files
- Compression
- Checksum & Verify
- Custom timestamp format
- Adjust `MaxTransferSize` and `BufferCount`

See <a href="https://sirsql.net/2012/12/13/20121212automated-backup-tuning/" data-href="https://sirsql.net/2012/12/13/20121212automated-backup-tuning/" title="https://sirsql.net/2012/12/13/20121212automated-backup-tuning/">https://sirsql.net/2012/12/13/20121212automated-backup-tuning/</a> for scripts to test your own backup performance

In [ ]:
$BackupParams = @{
    SqlInstance     = $Sql17;
    Path            = $BackupPath;
    Database        = "Satellites";
    CreateFolder    = $true;
    CopyOnly        = $true;
    Type            = "Full";
    FileCount       = 4;
    CompressBackup  = $true;
    Checksum        = $true;
    Verify          = $true;
    BufferCount     = 1000;
    MaxTransfersize = 65536;
    TimeStampFormat = "yyyy-MMM-dd HH.mm.ss";
}
$BackupResult = Backup-DbaDatabase @BackupParams;
$BackupResult | Format-List -Property *;
Get-ChildItem -Path "C:\SQL\Backup\Satellites" -Recurse | Sort-Object -Property LastWriteTime -Descending;


## What's Being Run?

The `-OutputScriptOnly` switch parameter tells `Backup-DbaDatabase` to not perform the backup but instead show the T-SQL to execute the backup.

In [ ]:
$BackupParams = @{
    SqlInstance      = $Sql17;
    Path             = $BackupPath;
    Database         = "Satellites";
    CreateFolder     = $true;
    CopyOnly         = $true;
    Type             = "Full";
    FileCount        = 4;
    CompressBackup   = $true;
    Checksum         = $true;
    Verify           = $true;
    BufferCount      = 1000;
    MaxTransfersize  = 65536;
    OutputScriptOnly = $true;
}
Backup-DbaDatabase @BackupParams;

## Restoring the Latest Backup
* The `-Path` parameter specifies a path to search for backups. If multiple backups are found, the most recent one will be used.
* `-Database` is the name the database will have when restored, not the original name of the database when it was backed up.

In [ ]:
$RestoreParams = @{
    SqlInstance = $Sql19;
    Path        = "C:\SQL\Backup\Satellites";
    Database    = "Satellites19";
}
$RestoreResult = Restore-DbaDatabase @RestoreParams;
$RestoreResult | Format-List -Property *;

## Restoring a Specific Backup
If the database name we're restoring to already exists, `-WithReplace` will overwrite it. **Use with caution!**

In [ ]:
$RestoreParams = @{
    SqlInstance = $Sql19;
    Path        = $SingleBackupFile;
    Database    = "Satellites19";
    WithReplace = $true;
}
$RestoreResult = Restore-DbaDatabase @RestoreParams;
$RestoreResult | Format-List -Property *;
Set-DbaDbOwner -SqlInstance $Sql19 -database Satellites19 -TargetLogin sa -Verbose;

## Just Looking (again)!

Maybe I'm just looking to learn how to construct a `RESTORE DATABASE` SQL statement. Other times, I just want to review the statement before it's executed.

The `-ReplaceDbNameInFile` renames the pysical files to match the database name when restored.

TODO: Move this lower Who hasn't run a bad update, right? Let's adjust a Stack Overflow user's reputation.

In [ ]:
$RestoreParams = @{
    SqlInstance      = $Sql19;
    Path             = "c:\SQL\Backup\Satellites"
    Database         = "Satellites19";
    WithReplace      = $true;
    OutputScriptOnly = $true;
    ReplaceDbNameInFile = $true;
}
Restore-DbaDatabase @RestoreParams;

## Point In Time Restore
Who hasn't run a bad update? Let's try adjusting a Stack Overflow user's reputation.

In [ ]:
$SOQueryParams = @{
    SqlInstance = "FLEXO\sql17";
    Database = "StackOverflow2010";
    Query = "select getdate() AS [QueryDate],* from [Users] where [DisplayName] = 'user46185';";
}
Invoke-DbaQuery @SOQueryParams;

In [ ]:
$SOUpdateParams = @{
    SqlInstance = "FLEXO\sql17";
    Database = "StackOverflow2010";
    Query = "update [Users] set [Reputation] = 200 where [DisplayName] = 'user461855';";
}
Invoke-DbaQuery @SOUpdateParams ;

Oops!

In [ ]:
$SOQueryParams = @{
    SqlInstance = "FLEXO\sql17";
    Database = "StackOverflow2010";
    Query = "select getdate() AS [QueryDate],* from [Users] where [DisplayName] in ('user46185','user461855');";
}
Invoke-DbaQuery @SOQueryParams;

Let's restore the database so we can fix the data

In [ ]:
# Take a log backup so we have something to work with
$BackupParams = @{
    SqlInstance = "FLEXO\sql17";
    Type = "Log";
    Database = "StackOverflow2010";
    Path = 'C:\sql\Backup\FLEXO$SQL17\StackOverflow2010\LOG';
}
Backup-Dbadatabase @BackupParams;

# Restore to another database
$RestoreParams = @{
    SqlInstance = "FLEXO\sql17";
    Path = 'C:\sql\Backup\FLEXO$SQL17\StackOverflow2010\';
    DatabaseName = "StackOverflow2010-Restored";
    RestoreTime = "2020-12-26 10:46:34";
    MaintenanceSolutionBackup = $true;
    ReplaceDbNameInFile = $true;
}
$RestoreResult = Restore-DbaDatabase @RestoreParams;
$RestoreResult | Format-List -Property *;

Database is restored, let's verify the data is in the right state

In [ ]:
$SOQueryParams = @{
    SqlInstance = "FLEXO\sql17";
    Database = "StackOverflow2010-Restored";
    Query = "select getdate() AS [QueryDate],* from [Users] where [DisplayName] in ('user46185','user461855');";
}
Invoke-DbaQuery @SOQueryParams;

Data looks good in the restored database, so we'll fix things up (not shown), then remove the restored database. I'm using `-Confirm:$false` because the prompt won't work in Azure Data Studio.

In [ ]:
Remove-DbaDatabase -SqlInstance FLEXO\SQL17 -Database StackOverflow2010-Restored -Confirm:$false;

# Basic Backup Setup
* Where do backups go?
* Do we have scheduled backups?
* Do they get compressed?

In [ ]:
Get-DbaDefaultPath -SqlInstance FLEXO\Sql17;

### Check & Set Backup Compression

In [ ]:
Get-DbaSpConfigure -SqlInstance FLEXO\Sql17 -Name DefaultBackupCompression;

In [ ]:
Set-DbaSpConfigure -SqlInstance FLEXO\Sql17 -Name DefaultBackupCompression -Value 1;

# Scheduled Backups
Manual backups are one thing, but we should be scheduling our backups to run regularly.

dbatools makes it easy to install & schedule backups with [Ola Hallengren's Maintenance Solution](https://ola.hallengren.com/).

## Installation
`Install-DbaMaintenanceSolution` retrieves the latest version from Github _or_ can use a locally-stored copy.

### Verifying Installation
Ola's scripts get installed with a categoy of "Database Maintenance" so we can filter the list of installed jobs.

## Installation

In [ ]:
$InstallParams = @{
    SqlInstance = "FLEXO\sql19";
    Solution    = "Backup";
    Database    = "DBAThings";
    CleanupTime = 25;
    InstallJobs = $true;
    LogToTable  = $true;
    ReplaceExisting = $true;
}
Install-DbaMaintenanceSolution @InstallParams;

In [ ]:
Get-DbaAgentJob -SqlInstance FLEXO\sql19 -Category "Database Maintenance" | Select-Object -Property Name;

### Check job info

Do the backup jobs have schedules assigned to them?

In [ ]:
$JobInfoParams = @{
    SqlInstance = $Sql19;
    Job         = @("DatabaseBackup - USER_DATABASES - Log", "DatabaseBackup - USER_DATABASES - Full");
}

Get-DbaAgentJob  @JobInfoParams | select-object Name, @{n = "ScheduleCount"; e = { $_.JobSchedules.Count } }

### Scheduling

Let's assign 5-minute and 15-minute schedules to our Log and Full backup jobs, respectively.

In [ ]:
$FiveMinuteParams = @{
    SqlInstance = "FLEXO\sql19";
    Schedule                = "Five Minutes";
    FrequencyType           = "Daily";
    FrequencyInterval       = 1;
    FrequencySubdayInterval = 5;
    FrequencySubdayType     = "Minutes";
    Force                   = $true;
}

$FifteenMinuteParams = @{
    SqlInstance = "FLEXO\sql19";
    Schedule                = "Fifteen Minutes";
    FrequencyType           = "Daily";
    FrequencyInterval       = 1;
    FrequencySubdayInterval = 15;
    FrequencySubdayType     = "Minutes";
    Force                   = $true;
}

$EveryFiveMinutes = New-DbaAgentSchedule @FiveMinuteParams;
$EveryFifteenMinutes = New-DbaAgentSchedule @FifteenMinuteParams;

$FullBackupParams = @{
    SqlInstance = "FLEXO\sql19";
    Job         = "DatabaseBackup - USER_DATABASES - FULL";
    Schedule    = $EveryFifteenMinutes;
}

$LogBackupParams = @{
    SqlInstance = "FLEXO\sql19";
    Job         = "DatabaseBackup - USER_DATABASES - LOG";
    Schedule    = $EveryFiveMinutes;
}

Set-DbaAgentJob @FullBackupParams;
Set-DbaAgentJob @LogBackupParams;
Start-DbaAgentJob -SqlInstance $Sql19 -Job "DatabaseBackup - USER_DATABASES - FULL";

# Checking on Backups
## Last Backup of Each Database

`Get-DbaLastBackup` retrieves the most recent backup of each database.

## Last Backup of Each Database

In [ ]:
Get-DbaLastBackup -SqlInstance FLEXO\sql17 | Format-Table -AutoSize;

## Backup History for One Database

In [ ]:
Get-DbaDbBackupHistory -SqlInstance FLEXO\sql17 -Database Satellites;

## Recent Backups

In [ ]:
$HistoryParams = @{
    SqlInstance     = FLEXO\sql17;
    IncludeCopyOnly = $true;
    Since           = (Get-Date).AddDays(-3);
    DeviceType      = "Disk";
}
Get-DbaDbBackupHistory @HistoryParams | Sort-Object -Property Start | Format-Table -AutoSize;

### Let's send that to Excel instead

In [ ]:
$HistoryParams = @{
    SqlInstance     = FLEXO\sql17;
    IncludeCopyOnly = $true;
    Since           = (Get-Date).AddDays(-3);
    DeviceType      = "Disk";
}
$BackupHistory = Get-DbaDbBackupHistory @HistoryParams;

$ExcelParams = @{
    Path         = "C:\users\andy\documents\BackupHistory.xlsx";
    ClearSheet   = $true;
    AutoSize     = $true;
    FreezeTopRow = $true;
    BoldTopRow   = $true;
    AutoFilter   = $true;
    Show         = $true;
}
$BackupHistory | Export-Excel @ExcelParams;

## Backup Speed
How fast are our backups?

In [ ]:
$ThroughputParams = @{
    SqlInstance = $Sql17;
}
$MeasurementFields = @(
     "SqlInstance"
     ,"Database"
     ,"MaxBackupDate"
    ,"AvgThroughput"
    ,"AvgDuration"
    ,"MinThroughput"
    ,"MaxThroughput"
    ,"BackupCount"
)
Measure-DbaBackupThroughput @ThroughputParams | Select-object -Property $MeasurementFields | Format-Table -AutoSize;

## Backup Integrity
Backups don't mean much if they can't be restored, right? How can we test that we have good, usable backups of our databases?

And then, how can we prove that we're doing it?

In [ ]:
$BackupTestParams = @{
    SqlInstance = "FLEXO\sql17";
    Destination = "FLEXO\sql19";
    Database=@("DBAThings","Geocaches","StackOverflow2010");
}
$BackupTestResults = Test-DbaLastBackup @BackupTestParams;
$BackupTestResults;

### DBCC History to Table

In [ ]:
# Backup test results to table here
#$BackupTestResults | Select-Object SourceServer,TestServer,Databasse,Size,RestoreResult,RbccResult,RestoreStart,RestoreElapsed,DbccStart,DbccElapsed, @{n="BackupDates";e={$_.BackupDates -join ";"}},@{n="BackupFiles";e={$_.BackupFiles -join ";"}}

$BackupTestResults | Convertto-DbaDataTable | write-dbadatatable -sqlinstance flexo\sql19 -database DBAThings -schema dbo -table BackupValidation -autocreatetable -usedynamicstringlength;

The auditors are coming! Provide documentation!

In [ ]:
$ExcelParams = @{
    Path         = "C:\users\andy\documents\BackupVerification.xlsx";
    ClearSheet   = $true;
    AutoSize     = $true;
    FreezeTopRow = $true;
    BoldTopRow   = $true;
    AutoFilter   = $true;
    Show         = $true;
}
invoke-dbaquery -sqlinstance FLEXO\sql19 -database DBAThings -query "select * from BackupValidation" |convertto-dbadatatable| Export-Excel @ExcelParams;

# Preparing for the Worst

`Export-DbaInstance`

In [ ]:
Stuff here

# Reset Environment

In [ ]:
Set-DbaSpConfigure -SqlInstance FLEXO\sql17 -Name DefaultBackupCompression -Value 0;
Remove-Item c:\users\andy\documents\BackupHistory.xlsx;
Remove-Item -Force -recurse -confirm:$false "C:\SQL\Backup\Satellites";
Remove-DbaDatabase -SqlInstance FLEXO\sql19 -Database Satellites19 -Confirm:$false;
Remove-DbaDatabase -SqlInstance FLEXO\sql17 -Database StackOverflow2010-Restored -Confirm:$false;
Restore-DbaDatabase -SqlInstance FLEXO\sql17 -DatabaseName StackOverflow2010 -ReplaceDbNameInFile -WithReplace -Path C:\Datasets\StackOverflow2010.bak;
# Remove Ola jobs from flexo\sql19
Get-DbaAgentJob -SqlInstance FLEXO\sql19 -Category "Database Maintenance" | Remove-DbaAgentJob;
(Get-DbaDbTable -SqlInstance flexo\sql19 -database dbathings|?{$_.name -in @("BackupValidation","CommandLog")}).DropIfExists();